In [4]:
##Import library
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
## Extract Fildelity Data

#Download data from multiple pages and fit into One table
frames = pd.DataFrame()
for i in range(0,5):    
    df = pd.read_html(f"https://research2.fidelity.com/fidelity/screeners/etf/public/etfholdings.asp?symbol=FMAT&view=Explore&sortBy=GICS_industry_name&sortDir=asc&page={i}", header=0)
    frames = frames.append(df)
    
frames.to_csv("fidelity_stocks.csv",header = True, index= False)

In [ ]:
## Blackrock data
#import os
#Download URL https://www.blackrock.com/investing/products/239503/?referrer=tickerSearch

#Renaming IYM to Blackrock
#os.rename('IYM_holdings.csv', 'Blackrock_holdings.csv')

br = pd.DataFrame(pd.read_csv("Blackrock_holdings.csv", encoding="cp1252"))


#remove rows 49,50,51 when analyzing
br = br.drop(br.index[[49,50,51]])

In [ ]:
## Vanguard Main Code

browser = webdriver.Chrome()
url = "https://investor.vanguard.com/etf/profile/overview/VAW/portfolio-holdings"
browser.get(url)

length = 121 #Number of data
full_table = pd.DataFrame(columns = ["Holdings", "Shares", "Market Value"],index = range(1, length))
row_marker = 0

for i in range(1,6):
    innerHTML = browser.execute_script("return document.body.innerHTML")
    element1  =  browser.find_element_by_id("vanguardProudctsScrimArea")
    HTML1 = element1.get_attribute('innerHTML')
    parsed_html1 = BeautifulSoup(HTML1, "lxml")
    table1 = parsed_html1.find("table")
    tag1 =  table1.find_all("tr")
    for row in range(1, len(tag1)-1):  
        column_marker = 0
        columns = tag1[row].find_all("span")
        for column in range(0, len(columns)): 
            if pd.isnull(columns[column].text): #check if empty then move on to next. else proceed
                continue
            else:    
                full_table.iat[row_marker, column_marker] = columns[column].text
                column_marker += 1
        row_marker += 1            
    #link_next_page = browser.find_element_by_css_selector(".goToNextPage()")
    link_next_page = browser.find_element_by_xpath('.//a[@data-ng-click="goToNextPage()"][contains(., "Next")]')
    link_next_page.click()
    time.sleep(4)


In [ ]:
## Save Vanguard table as csv
full_table.dropna(axis = 0).to_csv("Vanguard_data.csv", header = True, index = False)


## Load saved datasets

In [3]:
br = pd.read_csv("Blackrock_holdings.csv", encoding = "cp1252")
fid = pd.read_csv("fidelity_stocks.csv")
van = pd.read_csv("Vanguard_data.csv")


## Inspecting and cleaning data

In [4]:
br.head(10)
fid.head(10)
van.head(10)

#remove nan
br = br.dropna()
fid = fid.dropna()

#check the exchanges of stocks in blackrock
import collections
collections.Counter(list(br.Exchange))
br[["Issuer Ticker", "Name"]].loc[br["Exchange"] == "NASDAQ"]

,Issuer Ticker,Name,Asset Class,Weight (%),Price,Shares,Market Value,Notional Value,Sector,SEDOL,ISIN,Exchange
0,DWDP,DOWDUPONT INC,Equity,25.72,70.13,2212352.0,1.551522e+08,1.551522e+08,Materials,BYP66M4,US26078J1007,New York Stock Exchange Inc.
1,PX,PRAXAIR INC,Equity,7.18,158.19,273942.0,4.333488e+07,4.333488e+07,Materials,2699291,US74005P1049,New York Stock Exchange Inc.
2,ECL,ECOLAB INC,Equity,6.18,150.48,247536.0,3.724922e+07,3.724922e+07,Materials,2304227,US2788651006,New York Stock Exchange Inc.
3,APD,AIR PRODUCTS AND CHEMICALS INC,Equity,5.76,166.29,208952.0,3.474663e+07,3.474663e+07,Materials,2011602,US0091581068,New York Stock Exchange Inc.
4,LYB,LYONDELLBASELL INDUSTRIES NV CLASS,Equity,5.73,112.78,306477.0,3.456448e+07,3.456448e+07,Materials,B3SPXZ3,NL0009434992,New York Stock Exchange Inc.
5,PPG,PPG INDUSTRIES INC,Equity,4.36,110.54,237675.0,2.627259e+07,2.627259e+07,Materials,2698470,US6935061076,New York Stock Exchange Inc.
6,NUE,NUCOR CORP,Equity,3.14,62.50,303235.0,1.895219e+07,1.895219e+07,Materials,2651086,US6703461052,New York Stock Exchange Inc.
7,FCX,FREEPORT MCMORAN INC,Equity,2.99,14.05,1284426.0,1.804619e+07,1.804619e+07,Materials,2352118,US35671D8570,New York Stock Exchange Inc.
8,NEM,NEWMONT MINING CORP,Equity,2.62,31.03,508558.0,1.578055e+07,1.578055e+07,Materials,2636607,US6516391066,New York Stock Exchange Inc.
9,CE,CELANESE CORP SERIES A,Equity,2.51,116.83,129510.0,1.513065e+07,1.513065e+07,Materials,B05MZT4,US1508701034,New York Stock Exchange Inc.


,Symbol,Company,Sector,Industry,Geography,Market Cap.,Security Type,Weight
0,CASH,Cash†,--,--,--,--,Cash†,0.31%
1,USCR,US Concrete Inc,--,--,United States,$813.2M,Ordinary shares,0.09%
2,SMG,Scotts Miracle-Gro Co,Materials,Chemicals,United States,$4.2B,Ordinary shares,0.37%
3,SHW,Sherwin-Williams Co,Materials,Chemicals,United States,$43.2B,Ordinary shares,4.44%
4,TG,Tredegar Corp,Materials,Chemicals,United States,$713.6M,Ordinary shares,0.06%
5,WLK,Westlake Chemical Corp,Materials,Chemicals,United States,$11.9B,Ordinary shares,0.41%
6,OMN,OMNOVA Solutions Inc,Materials,Chemicals,United States,$428.5M,Ordinary shares,0.05%
7,OLN,Olin Corp,Materials,Chemicals,United States,$5.2B,Ordinary shares,0.60%
8,POL,PolyOne Corp,Materials,Chemicals,United States,$3.4B,Ordinary shares,0.39%
9,RYAM,Rayonier Advanced Materials Inc,Materials,Chemicals,United States,$1.1B,Ordinary shares,0.13%


,Holdings,Shares,Market Value
0,DowDuPont Inc.,"8,542,881","$587,493,926"
1,Praxair Inc.,"1,053,213","$176,413,178"
2,Sherwin-Williams Co.,"310,306","$136,761,163"
3,LyondellBasell Industries NV Class A,"1,230,100","$136,282,779"
4,Ecolab Inc.,"953,708","$134,186,716"
5,Air Products & Chemicals Inc.,"803,136","$131,850,837"
6,PPG Industries Inc.,"916,673","$101,439,034"
7,Freeport-McMoRan Inc.,"5,044,771","$83,238,722"
8,Nucor Corp.,"1,165,710","$78,020,970"
9,International Paper Co.,"1,438,984","$77,316,610"


Counter({'-': 2, 'NASDAQ': 2, 'New York Stock Exchange Inc.': 47})

,Issuer Ticker,Name
14,STLD,STEEL DYNAMICS INC
26,RGLD,ROYAL GOLD INC


In [ ]:
br.columns
list(fid)
list(van)

In [ ]:
type(br.Name[0])
type(fid.Company[0])
type(van.Holdings[0])

In [ ]:
#Same companies in different datasets listed differently.Ex. "DowDupont" vs "Dow Dupont"
#Need to modify to be consistent. Condense all spaces and change to all lower string to match and strip all dots

In [72]:
## Matching companies that are present in all 3 portfolios

Match_table = pd.DataFrame(columns = ["Name", "Ticker", "Increase from start 2010", "Increase from mid 2010", "Latest Price"])

for i in range(0, len(br)): #since first argument of re.search must be list 
    for k in range(0,len(fid)):
        if "".join(br.Name[i].split()).lower().strip(".") == "".join(fid.Company[k].split()).lower().strip("."):
            for j in range(0,len(van)):
                if "".join(br.Name[i].split()).lower().strip(".") == "".join(van.Holdings[j].split()).lower().strip("."):
                        #Match_list.append(br.Name[i])
                        Match_table.loc[len(Match_table)] = [br.Name[i], br["Issuer Ticker"][i], 0, 0,0]
                else: continue

Match_table
            

,Name,Ticker,Increase from start 2010,Increase from mid 2010,Latest Price
0,DOWDUPONT INC,DWDP,0,0,0
1,PRAXAIR INC,PX,0,0,0
2,ECOLAB INC,ECL,0,0,0
3,PPG INDUSTRIES INC,PPG,0,0,0
4,NUCOR CORP,NUE,0,0,0
5,NEWMONT MINING CORP,NEM,0,0,0
6,CF INDUSTRIES HOLDINGS INC,CF,0,0,0
7,CF INDUSTRIES HOLDINGS INC,CF,0,0,0
8,FMC CORP,FMC,0,0,0
9,STEEL DYNAMICS INC,STLD,0,0,0


In [73]:
# Drop duplicates
Match_table = Match_table.groupby('Name').apply(lambda x: x.sort_values('Ticker')).reset_index(drop=True)
Match_table = Match_table.drop_duplicates(subset=["Name"], keep='first')
Match_table = Match_table.reset_index(drop=True)


In [74]:
#drop companies not in public listing since 2010
L = []
remove_list = sorted(["AXTA","GCP", "NGVT", "PAH", "VVV"])
for i in remove_list:
       L.append(Match_table[Match_table["Ticker"] == i].index[0])

Match_tableV1 = Match_table.drop(Match_table.index[L]).reset_index(drop=True)
#Match_tableV1


,Name,Ticker,Increase from start 2010,Increase from mid 2010,Latest Price
0,ALBEMARLE CORP,ALB,0,0,0
1,ALCOA CORP,AA,0,0,0
2,ALLEGHENY TECHNOLOGIES INC,ATI,0,0,0
3,CABOT CORP,CBT,0,0,0
4,CARPENTER TECHNOLOGY CORP,CRS,0,0,0
5,CF INDUSTRIES HOLDINGS INC,CF,0,0,0
6,COMPASS MINERALS INTERNATIONAL INC,CMP,0,0,0
7,DOMTAR CORP,UFS,0,0,0
8,DOWDUPONT INC,DWDP,0,0,0
9,ECOLAB INC,ECL,0,0,0


In [75]:
#Get Data of NYSE stocks

############## Disable notifications
#from selenium.webdriver.chrome.options import Options

#options = Options()
#options.add_argument("--disable-notifications")
#browser = webdriver.Chrome(chrome_options=options)
##############

browser = webdriver.Chrome()
x = 0

for i in list(Match_tableV1.Ticker):
    
    while i not in ["STLD","RGLD"]:
        browser.get(f"https://www.nyse.com/quote/XNYS:{i}");
        
        #Execute, read, and parse into string
        innerHTML = browser.execute_script("return document.body.innerHTML");
        #element = browser.find_element_by_class_name("d-widget d-vbox d-flex1 DataTable-nyse");
        element = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]');
        HTML = element.get_attribute('innerHTML');
        parsed_html = BeautifulSoup(HTML, "lxml");
    
        #Close dialog "user notice". Occurs only the first time accessing the page
        while x < 1:
            dialog_button = browser.find_element_by_xpath('/html/body/div[8]/div[2]/div/div/div[2]/button');
            dialog_button.click();
            x += 1;
    
        #Drop down button to change periods
        drop_down = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[2]/span/div[1]/div/div/button');
        browser.execute_script("arguments[0].click()", drop_down);
        #drop_down = browser.find_element_by_xpath('//button[contains(@class, "d-button-normal d-button-dropdown") and contains(text(), "Daily")]')
   
    
        #Select third element from drop down menu
        #select_monthly = browser.find_element_by_xpath('.//li[@data-index = "2"][contains(., "Monthly")]')
        select_monthly = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[2]/span/div[1]/div/div/ul/li[3]');
        browser.execute_script("arguments[0].click()", select_monthly);
    
        #enter start date to beginning of 2010
        inputDate= browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[2]/span/div[2]/div[1]/div/div/div/input');
        inputDate.send_keys(Keys.CONTROL + "a");
        inputDate.send_keys(Keys.DELETE);
        inputDate.send_keys("2010-01-01");
        inputDate.send_keys(Keys.ENTER);
    
        #CLick Go button
        browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[2]/span/div[2]/button/span').click();
    
        #Get latest price
        latest_price = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[3]/div/div/div[2]/div/div[1]/div[2]/div');
        latest = float(latest_price.text);
    
        #Get start 2010 price
        start_price = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[3]/div/div/div[2]/div/div[105]/div[2]/div');
        start = float(start_price.text);
    
        #Get mid 2010 price
        mid_price = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[3]/div/div/div[2]/div/div[100]/div[2]/div');
        mid = float(mid_price.text);
    
        #Percent increase 
        Increase_start = latest/start ;
        Increase_mid = latest/mid ;
    
        #Ticker table index number
        num = Match_tableV1[Match_tableV1["Ticker"] == i].index[0] ;
    
        #update table
        #Match_table.iat[num, "Increase from start 2010"] = Increase_start;
        #Match_table.iat[num, "Increase from mid 2010"] = Increase_mid ;
        Match_tableV1.iat[num, 2] = Increase_start;
        Match_tableV1.iat[num, 3] = Increase_mid ;
        Match_tableV1.iat[num, 4] = latest
    
        break
    
    
 
print(Match_tableV1, flush = True)

                                  Name Ticker Increase from start 2010  \
0                       ALBEMARLE CORP    ALB                  2.62801   
1                           ALCOA CORP     AA                  1.18454   
2           ALLEGHENY TECHNOLOGIES INC    ATI                 0.579362   
3                           CABOT CORP    CBT                  2.44583   
4            CARPENTER TECHNOLOGY CORP    CRS                  2.13793   
5           CF INDUSTRIES HOLDINGS INC     CF                  2.80712   
6   COMPASS MINERALS INTERNATIONAL INC    CMP                 0.915129   
7                          DOMTAR CORP    UFS                  1.80756   
8                        DOWDUPONT INC   DWDP                   2.4734   
9                           ECOLAB INC    ECL                  3.34466   
10                            FMC CORP    FMC                  2.98568   
11                       HUNTSMAN CORP    HUN                  2.62132   
12   KAPSTONE PAPER AND PACKAGING CORP

In [76]:
#Get Data of NASDAQ stocks

x = 0
browser = webdriver.Chrome()

for i in ["RGLD","STLD"]:
    
        browser.get(f"https://www.nyse.com/quote/XNGS:{i}");
        
        #Execute, read, and parse into string
        innerHTML = browser.execute_script("return document.body.innerHTML");
        #element = browser.find_element_by_class_name("d-widget d-vbox d-flex1 DataTable-nyse");
        element = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]');
        HTML = element.get_attribute('innerHTML');
        parsed_html = BeautifulSoup(HTML, "lxml");
    
        #Close dialog "user notice" occurs only the first time accessing the page
        while x < 1:
            dialog_button = browser.find_element_by_xpath('/html/body/div[8]/div[2]/div/div/div[2]/button');
            dialog_button.click();
            x += 1;
    
        #Drop down button to change periods
        drop_down = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[2]/span/div[1]/div/div/button');
        browser.execute_script("arguments[0].click()", drop_down);
        #drop_down = browser.find_element_by_xpath('//button[contains(@class, "d-button-normal d-button-dropdown") and contains(text(), "Daily")]')
   
    
        #Select third element from drop down menu
        #select_monthly = browser.find_element_by_xpath('.//li[@data-index = "2"][contains(., "Monthly")]')
        select_monthly = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[2]/span/div[1]/div/div/ul/li[3]');
        browser.execute_script("arguments[0].click()", select_monthly);
    
        #enter start date to beginning of 2010
        inputDate= browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[2]/span/div[2]/div[1]/div/div/div/input');
        inputDate.send_keys(Keys.CONTROL + "a");
        inputDate.send_keys(Keys.DELETE);
        inputDate.send_keys("2010-01-01");
        inputDate.send_keys(Keys.ENTER);
    
        #CLick Go button
        browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[2]/span/div[2]/button/span').click();
    
        #Get latest price
        latest_price = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[3]/div/div/div[2]/div/div[1]/div[2]/div');
        latest = float(latest_price.text);
    
        #Get start 2010 price
        start_price = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[3]/div/div/div[2]/div/div[105]/div[2]/div');
        start = float(start_price.text);
    
        #Get mid 2010 price
        mid_price = browser.find_element_by_xpath('//*[@id="content-9d1f8b01-08a6-4db5-99fa-c40f5873615a"]/div/div[1]/div/div[2]/div[1]/div[4]/div[3]/div/div/div[2]/div/div[100]/div[2]/div');
        mid = float(mid_price.text);
    
        #Percent increase 
        Increase_start = latest/start ;
        Increase_mid = latest/mid ;
    
        #Ticker table index number
        num = Match_tableV1[Match_tableV1["Ticker"] == i].index[0] ;
    
        #update table
        #Match_table.iat[num, "Increase from start 2010"] = Increase_start;
        #Match_table.iat[num, "Increase from mid 2010"] = Increase_mid ;
        Match_tableV1.iat[num, 2] = Increase_start;
        Match_tableV1.iat[num, 3] = Increase_mid ;
        Match_tableV1.iat[num, 4] = latest
 
print(Match_tableV1, flush = True)

                                  Name Ticker Increase from start 2010  \
0                       ALBEMARLE CORP    ALB                  2.62801   
1                           ALCOA CORP     AA                  1.18454   
2           ALLEGHENY TECHNOLOGIES INC    ATI                 0.579362   
3                           CABOT CORP    CBT                  2.44583   
4            CARPENTER TECHNOLOGY CORP    CRS                  2.13793   
5           CF INDUSTRIES HOLDINGS INC     CF                  2.80712   
6   COMPASS MINERALS INTERNATIONAL INC    CMP                 0.915129   
7                          DOMTAR CORP    UFS                  1.80756   
8                        DOWDUPONT INC   DWDP                   2.4734   
9                           ECOLAB INC    ECL                  3.34466   
10                            FMC CORP    FMC                  2.98568   
11                       HUNTSMAN CORP    HUN                  2.62132   
12   KAPSTONE PAPER AND PACKAGING CORP

In [77]:
## Save table as csv
Match_tableV1.to_csv("Matched_data_table.csv", header = True, index = False)

In [ ]:
## Find those whose price has been up >380% since 2010 and more than $90 today

In [9]:
df = pd.read_csv("Matched_data_table.csv")

cond1 = df["Increase from start 2010"] >3.8
cond2 = df["Increase from mid 2010"] >3.8
cond3 = df["Latest Price"]>90

df[cond1 & cond3]
df[cond2 & cond3]

,Name,Ticker,Increase from start 2010,Increase from mid 2010,Latest Price
25,WESTLAKE CHEMICAL CORP,WLK,7.887232,9.562697,94.03


,Name,Ticker,Increase from start 2010,Increase from mid 2010,Latest Price
14,NEWMARKET CORP,NEU,3.433551,3.931773,399.94
25,WESTLAKE CHEMICAL CORP,WLK,7.887232,9.562697,94.03
